# Acquire (acquire.py)

## Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

### acquire & summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
    - ***Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for `airconditioningtypeid`.***
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

In [7]:
import pandas as pd
import numpy as np
import os
from env import host, user, password

###################### Acquire Zillow Data ######################

def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
def new_zillow_data():
    '''
    This function reads the zillow data from the Codeup db into a df,
    write it to a csv file, and returns the df.
    '''
    sql_query = '''
            SELECT *
            FROM properties_2017 as prop 
            INNER JOIN (
                    SELECT id, p.parcelid, logerror, transactiondate
                    FROM predictions_2017 AS p
                    INNER JOIN (
                            SELECT parcelid,  MAX(transactiondate) AS max_date
                            FROM predictions_2017 
                            GROUP BY (parcelid)
                    ) AS sub
                        ON p.parcelid = sub.parcelid
                    WHERE p.transactiondate = sub.max_date
            ) AS subq
                ON prop.id = subq.id;
                 '''
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df.to_csv('zillow.csv')
    return df

def get_zillow_data(cached=False):
    '''
    This function reads in titanic data from Codeup database if cached == False 
    or if cached == True reads in zillow df from a csv file, returns df
    '''
    if cached or os.path.isfile('zillow_df.csv') == False:
        df = new_zillow_data()
    else:
        df = pd.read_csv('zillow_df.csv', index_col=0)
    return df

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [8]:
df = get_zillow_data()

In [9]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,parcelid,logerror,transactiondate
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,2016.0,9.0,NaN,None,NaN,NaN,0,14297519,0.025595,2017-01-01
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,2015.0,27516.0,NaN,None,NaN,NaN,1,17052889,0.055619,2017-01-01
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,2016.0,774261.0,20800.37,None,NaN,NaN,2,14186244,0.005383,2017-01-01
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,2016.0,594416.0,14557.57,None,NaN,NaN,3,12177905,-0.103410,2017-01-01
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,2016.0,243350.0,5725.17,None,NaN,NaN,4,10887214,0.006940,2017-01-01


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77414 non-null  int64  
 1   parcelid                      77414 non-null  int64  
 2   airconditioningtypeid         21395 non-null  float64
 3   architecturalstyletypeid      151 non-null    float64
 4   basementsqft                  44 non-null     float64
 5   bathroomcnt                   77414 non-null  float64
 6   bedroomcnt                    77414 non-null  float64
 7   buildingclasstypeid           381 non-null    float64
 8   buildingqualitytypeid         50182 non-null  float64
 9   calculatedbathnbr             74202 non-null  float64
 10  decktypeid                    510 non-null    float64
 11  finishedfloor1squarefeet      5953 non-null   float64
 12  calculatedfinishedsquarefeet  76306 non-null  float64
 13  f

In [11]:
df.describe()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id,parcelid,logerror
count,77414.000000,7.741400e+04,21395.000000,151.000000,44.000000,77414.000000,77414.000000,381.000000,50182.000000,74202.000000,...,7.615000e+04,7.649000e+04,77414.000000,7.580800e+04,7.690500e+04,1502.000000,7.532400e+04,77414.000000,7.741400e+04,77414.000000
mean,38848.445307,1.337669e+07,1.926899,7.430464,601.613636,2.214632,3.084106,3.774278,6.300666,2.309661,...,1.787250e+05,4.405307e+05,2015.998217,2.649626e+05,5.368690e+03,13.920772,6.048976e+13,38848.445307,1.300670e+07,0.016578
std,22404.198916,8.324811e+06,3.132454,2.629347,462.680774,1.083171,1.275663,0.460514,1.726176,1.001730,...,4.021004e+05,8.339476e+05,0.068170,5.284760e+05,1.040392e+04,1.528231,2.100220e+11,22404.198916,3.480690e+06,0.170342
min,0.000000,1.071174e+07,1.000000,2.000000,63.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,5.000000e+00,8.000000e+00,2003.000000,4.000000e+00,2.540000e+00,6.000000,6.037101e+13,0.000000,1.071186e+07,-4.655420
25%,19459.250000,1.158595e+07,1.000000,7.000000,222.000000,2.000000,2.000000,4.000000,5.000000,2.000000,...,7.833650e+04,1.870282e+05,2016.000000,7.840075e+04,2.449780e+03,14.000000,6.037311e+13,19459.250000,1.153834e+07,-0.024387
50%,38871.500000,1.253280e+07,1.000000,7.000000,542.000000,2.000000,3.000000,4.000000,6.000000,2.000000,...,1.283935e+05,3.214315e+05,2016.000000,1.753420e+05,4.005480e+03,14.000000,6.037572e+13,38871.500000,1.253072e+07,0.006624
75%,58254.750000,1.411519e+07,1.000000,7.000000,744.500000,3.000000,4.000000,4.000000,8.000000,3.000000,...,2.055275e+05,5.129815e+05,2016.000000,3.246770e+05,6.198450e+03,15.000000,6.059042e+13,58254.750000,1.421149e+07,0.039211
max,77613.000000,1.696019e+08,13.000000,21.000000,2443.000000,20.000000,25.000000,5.000000,12.000000,20.000000,...,6.640493e+07,1.491392e+08,2016.000000,9.401108e+07,1.824155e+06,15.000000,6.111009e+13,77613.000000,1.676893e+08,5.262999


3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an attribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [72]:
def missing_rows_df(df):
    '''Takes in a dataframe of observations and attributes 
    and returns a dataframe where each row is an attribute name, 
    the first column is the number of rows with missing values 
    for that attribute, and the second column is percent of total 
    rows that have missing values for that attribute'''
    d = {'num_rows_missing': df.isna().sum(), 
         'pct_rows_missing': df.isna().sum()/len(df)}
    new_df = pd.DataFrame(data=d)
    return new_df

In [73]:
df_mr = missing_rows_df(df)
df_mr.head()

,num_rows_missing,pct_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,56019,0.723629
architecturalstyletypeid,77263,0.998049
basementsqft,77370,0.999432


In [83]:
cols_no_missing = df_mr[df_mr.num_rows_missing == 0].T.columns.to_list()

In [90]:
cols_more_than_50pct_missing = df_mr[df_mr.pct_rows_missing > .5].T.columns.to_list()

takeaways: 

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [109]:
pd.DataFrame(df.isna().sum() == len(df))

,0
id,False
parcelid,False
airconditioningtypeid,False
architecturalstyletypeid,False
basementsqft,False
...,...
censustractandblock,False
id,False
parcelid,False
logerror,False


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77414 non-null  int64  
 1   parcelid                      77414 non-null  int64  
 2   airconditioningtypeid         21395 non-null  float64
 3   architecturalstyletypeid      151 non-null    float64
 4   basementsqft                  44 non-null     float64
 5   bathroomcnt                   77414 non-null  float64
 6   bedroomcnt                    77414 non-null  float64
 7   buildingclasstypeid           381 non-null    float64
 8   buildingqualitytypeid         50182 non-null  float64
 9   calculatedbathnbr             74202 non-null  float64
 10  decktypeid                    510 non-null    float64
 11  finishedfloor1squarefeet      5953 non-null   float64
 12  calculatedfinishedsquarefeet  76306 non-null  float64
 13  f